In [1]:
import pybaseball as pb 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
from pybaseball import statcast

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
gbg = pb.retrosheet.season_game_logs(2008)

In [58]:
seasons.pop()

2019

In [59]:
seasons

[2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018]

In [60]:
def season_pull(year):
    name = 'season_'+str(year)
    '''Pull from retrosheet for season indicated by year 
    EX: season_pull(2008) returns a DataFrame containing 
    the entire game by game for the 2008 MLB Season'''
    df=pb.retrosheet.season_game_logs(year)
    df.to_csv('C:/Users/dakot/Documents/GitHub/baseball_predict/Data/'+name+'.csv' )
    return df 

In [61]:
for x in seasons:
    season_pull(x)

In [11]:
seasons = []
for x in range(2008,2020,1):
    seasons.append(x)